# Music Recomendations

In [52]:
from datasets.members import MembersDataset
from datasets.songs import SongsDataset
from datasets.train import TrainDataset
from datasets.songs_info import SongsInfoDataset

from models.catboost_model import CatBoostModel
from models.embeddings_model import EmbeddingModel

from plot import * 
from utils import * 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Подготовка данных

In [20]:
random_state = 42

Загрузим данные в память. 

In [21]:
%%time
members_dataset = MembersDataset.from_path("./data/members.csv")
songs_dataset = SongsDataset.from_path("./data/songs.csv")
train_dataset = TrainDataset.from_path("./data/train.csv")
songs_info_dataset = SongsInfoDataset.from_path("./data/song_extra_info.csv")

CPU times: user 56.3 s, sys: 1.01 s, total: 57.3 s
Wall time: 57.5 s


Построим признаки для песен и пользователей и посмотрим, что получилось. 

In [22]:
%%time
members_dataset_featured = members_dataset.create_features(inplace=False)
songs_dataset_featured = songs_dataset.create_features(songs_info_dataset, inplace=False)

CPU times: user 11.1 s, sys: 460 ms, total: 11.6 s
Wall time: 11.7 s


In [24]:
members_dataset_featured.pandas_df.head(3)

,msno,city,gender,registered_via,bd_category,registration_init_year,expiration_date_year
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,<UNK>,7,<UNK>,2011,2017
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,<UNK>,7,<UNK>,2015,2017
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,<UNK>,4,<UNK>,2016,2017


In [30]:
songs_dataset_featured.pandas_df.head(3)

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language,genres_count,artist_name_count,composer_count,lyricists_count,isrc_year
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0,1,1,1,1,2014
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0,1,1,3,1,-1
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,<UNK>,<UNK>,31.0,1,1,0,0,-1


Теперь добавим в обучающую выборку признаков из песен и пользователей. 

In [35]:
train_dataset = train_dataset.merge(members_dataset_featured, on="msno", how="left")
train_dataset = train_dataset.merge(songs_dataset_featured, on="song_id", how="left")

In [36]:
train_dataset.pandas_df.head(3)

,msno,song_id,source_system_tab,source_screen_name,source_type,target,city,gender,registered_via,bd_category,...,genre_ids,artist_name,composer,lyricist,language,genres_count,artist_name_count,composer_count,lyricists_count,isrc_year
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,1,<UNK>,7,<UNK>,...,359,Bastille,Dan Smith| Mark Crew,<UNK>,52.0,1.0,1.0,2.0,0.0,2016
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1,13,female,9,young,...,1259,Various Artists,<UNK>,<UNK>,52.0,1.0,1.0,0.0,0.0,1999
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1,13,female,9,young,...,1259,Nas,N. Jones、W. Adams、J. Lordan、D. Ingle,<UNK>,52.0,1.0,1.0,1.0,0.0,2006


Теперь преобразуем некоторые колонки категориальным значениям и заполнил пропуски. 

In [37]:
train_dataset.to_category(["msno", "song_id"])
train_dataset.fill_na_category(["source_system_tab", "source_screen_name", "source_type"])

In [38]:
train_dataset.pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7377418 entries, 0 to 7377417
Data columns (total 23 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   msno                    category
 1   song_id                 category
 2   source_system_tab       category
 3   source_screen_name      category
 4   source_type             category
 5   target                  uint8   
 6   city                    category
 7   gender                  category
 8   registered_via          category
 9   bd_category             category
 10  registration_init_year  int64   
 11  expiration_date_year    int64   
 12  song_length             float64 
 13  genre_ids               category
 14  artist_name             category
 15  composer                category
 16  lyricist                category
 17  language                category
 18  genres_count            float64 
 19  artist_name_count       float64 
 20  composer_count          float64 
 21  lyricist

Все круто!

In [39]:
na_mask = train_dataset.pandas_df.isna().any(axis=1)
print(f"Количество записей, для которых не нашлось пользователя или песни: {sum(na_mask)}")

Количество записей, для которых не нашлось пользователя или песни: 114


Их оказалось не так много, уберем из нашего набора данных.

In [40]:
train_dataset = train_dataset.remove_by_mask(na_mask)

Так как **CatBoost** на CPU обучается достаточно долго (больше часа), то было решено обучать на GPU. Однако GPU не поддерживает группы больше 1023, поэтому тренировочный набор данных был уменьшен. При этом, желательно сохранить порядок записей.

In [41]:
train_dataset_reduced = train_dataset.reduce_by_members(size=1023)

In [42]:
train_dataset_reduced.pandas_df.target.value_counts()

1    3564182
0    3451109
Name: target, dtype: int64

Соотношение классов осталось тем же, все хорошо. 

In [43]:
print(f"Изменение размеров: {len(train_dataset)} -> {len(train_dataset_reduced)}")

Изменение размеров: 7377304 -> 7015291


## Задание 1

Построить рекомендации для каждого пользователя, произвести оценку качества с помощью 5-fold CV с метриками NDCG, ROC AUC per user.

При разработке признаков я умеренно не стал брать признаки, полученные с помощью матричных факторизаций, чтобы потом добавить неклассические эмбеддинги, полученные в задании 2. 

Для **CatBoost** необоходимо, чтобы данные были упорядочены по **queries**, поэтому будем обучать модели на упорядоченных данных, но эмбеддинги составлять на основе натурального порядка. 

In [ ]:
train_dataset_reduced_sorted = train_dataset_reduced.sort_by("msno")
model = CatBoostModel("YetiRank", 150, "CPU", random_state)
scores = model.cv_scores(train_dataset_reduced_sorted, n_splits=5)

Train dataset size: 5612232 | Test dataset size: 1403059
Groupwise loss function. OneHotMaxSize set to 10
0:	total: 19.7s	remaining: 48m 51s
1:	total: 34s	remaining: 41m 56s
2:	total: 48.1s	remaining: 39m 15s
3:	total: 1m 2s	remaining: 38m 6s
4:	total: 1m 16s	remaining: 36m 57s
5:	total: 1m 32s	remaining: 36m 59s
6:	total: 1m 48s	remaining: 36m 50s
7:	total: 2m 3s	remaining: 36m 28s
8:	total: 2m 19s	remaining: 36m 18s
9:	total: 2m 33s	remaining: 35m 52s
10:	total: 2m 48s	remaining: 35m 28s
11:	total: 3m 2s	remaining: 34m 53s
12:	total: 3m 16s	remaining: 34m 31s
13:	total: 3m 32s	remaining: 34m 21s
14:	total: 3m 47s	remaining: 34m 4s
15:	total: 4m 2s	remaining: 33m 47s
16:	total: 4m 17s	remaining: 33m 35s
17:	total: 4m 31s	remaining: 33m 9s
18:	total: 4m 45s	remaining: 32m 51s
19:	total: 5m 1s	remaining: 32m 41s
20:	total: 5m 17s	remaining: 32m 32s
21:	total: 5m 33s	remaining: 32m 21s
22:	total: 5m 46s	remaining: 31m 55s
23:	total: 6m 1s	remaining: 31m 36s
24:	total: 6m 16s	remaining: 3

## Задание 2